# Note

In this notebook, I show how to use eg.py.

# Importing some libraries

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
import eg
import igetfile

# Downloading analyzed file

In [7]:
# define some variables
shotnum = 120724 # shotnumber
diag = 'tsmap'   # diagname
filename = 'tsmap.dat' # local filename 

# downloading file
ts_filename = igetfile.igetfile(diag, shotnum, 1, filename)

data/tsmap/120000/120724/000001/tsmap@120724.dat.zip
tsmap@120724.dat.zip


In [8]:
print ts_filename

tsmap.dat


# Reading the downloaded file

In [9]:
# actual download
tsdata = eg.loadtxt(ts_filename)

# Member methods and variables in eg